# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

import seaborn as sns

from sklearn.model_selection import train_test_split

from azureml.train.automl import AutoMLConfig

from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="capstone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
cpu_cluster_name = "udacity-project3"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
dataset_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
dataset = Dataset.Tabular.from_delimited_files(path=dataset_link, header=False).to_pandas_dataframe()
dataset.columns =['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'iris_class'] 

dataset.head()

In [ ]:
dataset.describe()

In [ ]:
plt.title("Sepal Plot")
sns.scatterplot('sepal_length', 'sepal_width', data=dataset, hue='Species', palette="deep")
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 1))
plt.show()

In [ ]:
plt.title("Petal Plot")
sns.scatterplot('petal_length', 'petal_width', data=dataset, hue='Species', palette="deep")
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 1))
plt.show()

In [ ]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    'experiment_timeout_minutes': 15,
    'primary_metric' : 'accuracy',
    'n_cross_validations': 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train,
                             label_column_name='iris_class', 
                             enable_early_stopping= True,
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

best_run

In [ ]:
best_model

In [ ]:
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])
print(best_model._final_estimator)

In [ ]:
#TODO: Save the best model
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
print(model_name)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_run.register_model(model_name = 'automl_model', 
                                                model_path='outputs/automl.joblib')

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True)  

service_name = 'service_aml'
service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
test_ws = train.tail()
target = test_ws['iris_class']
test_ws = test_ws[:-1]

test_json = json.dumps({'data':test_ws.to_dict(orient='records')})
print(test_json)

results = service.run(teste_json)
print('Model results:', results)
print('Actual results:', y_actual.values)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

service.delete()